# Twitter Scrapper --- This scrapper requires user's credentials for login

In [32]:
# import required libraries
import csv
import re
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver import Chrome

In [33]:
driver = Chrome() # selenium webdriver instant

In [34]:
driver.get('https://www.twitter.com/login') # login page instance

In [35]:
# This code contains user's username already
username = driver.find_element_by_xpath('//input[@name="text"]')
username.send_keys('@gk_woode') # username can be changed from this line of code
username.send_keys(Keys.RETURN)

In [36]:
my_password = getpass() # password instance

········


In [37]:
# This code picks user's entered password to login
password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

In [38]:
# Code searches for particular tweet ||| but here it searches for tweets on ghanacard
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('#ghanacard') # tweet can be changed from this line of code

In [39]:
search_input.send_keys(Keys.RETURN)

In [40]:
driver.find_element_by_link_text('Latest').click()

In [41]:
cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

In [42]:
card = cards[0]

In [43]:
#username
card.find_element_by_xpath('.//span').text

'Martin Asiedu-Dartey'

In [44]:
#handle
card.find_element_by_xpath('.//span[contains(text(), "@")]').text

'@NewsyMartin'

In [45]:
#date posted
card.find_element_by_xpath('.//time').get_attribute('datetime')

'2022-02-22T17:23:32.000Z'

In [46]:
#tweet
comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
responds = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
comment + responds

"Martin Asiedu-Dartey\n@NewsyMartin\n·\n17hFinally, I've got my Ghana Card . Have you got yours yet?\n#GhanaCard \n2\n1\n6"

In [47]:
#replies
card.find_element_by_xpath('.//div[@data-testid="reply"]').text

'2'

In [48]:
#retweets
card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

'1'

In [49]:
#likes
card.find_element_by_xpath('.//div[@data-testid="like"]').text

'6'

In [50]:
#define function
# Function to extract data from tweet card

def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
#     try:
#         handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
#     except NoSuchElementException:
#         return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responds = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responds
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet

In [51]:
get_tweet_data(card)

('Martin Asiedu-Dartey',
 '@NewsyMartin',
 '2022-02-22T17:23:32.000Z',
 "Martin Asiedu-Dartey\n@NewsyMartin\n·\n17hFinally, I've got my Ghana Card . Have you got yours yet?\n#GhanaCard \n2\n1\n6",
 '2',
 '1',
 '6')

In [52]:
tweet_data = []

# cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

In [53]:
tweet_data[0]

('Martin Asiedu-Dartey',
 '@NewsyMartin',
 '2022-02-22T17:23:32.000Z',
 "Martin Asiedu-Dartey\n@NewsyMartin\n·\n17hFinally, I've got my Ghana Card . Have you got yours yet?\n#GhanaCard \n2\n1\n6",
 '2',
 '1',
 '6')

In [54]:
# driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [55]:
# getting tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

In [56]:
while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-10:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
                
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

In [57]:
len(data)

599

## Save the tweet data

In [58]:
with open('ghanacard.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header =['Username', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)